# 制約最適化システム SCOP

>  Coonstraint Programming Solver SCOP 

SCOP（Solver forCOnstraint Programing：スコープ）は，
大規模な制約最適化問題を高速に解くためのソルバーである．

ここで，制約最適化(constraint optimization)とは，
数理最適化を補完する最適化理論の体系であり，
組合せ最適化問題に特化した求解原理-メタヒューリスティクス(metaheuristics)-を用いるため，
数理最適化ソルバーでは求解が困難な大規模な問題に対しても，効率的に良好な解を探索することができる．

SCOPのトライアルバージョンは， http://logopt.com/scop2/ からダウンロードもしくはgithub https://github.com/scmopt/scop からクローンできる．
また，テクニカルドキュメントは， https://scmopt.github.io/manual/14scop.html にある．

In [ ]:
#| default_exp scop

In [ ]:
#| export

#Pydantic
from typing import List, Optional, Union, Tuple, Dict, Set, Any, DefaultDict, ClassVar
from pydantic import (BaseModel, Field, ValidationError, validator, 
                      confloat, conint, constr, Json, PositiveInt, NonNegativeInt)
from pydantic.tools import parse_obj_as
from datetime import datetime, date, time

import os
import sys
import re
import copy
import platform
import string
_trans = str.maketrans(":-+*/'(){}^=<>$ |#?,\¥", "_"*22) #文字列変換用
import ast
import pickle
import datetime as dt
from collections import Counter
import pathlib

#以下非標準ファイル
#import pandas as pd
#import numpy as np
import plotly.graph_objs as go
import plotly
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

## 重み付き制約充足問題

ここでは，SCOPで対象とする重み付き制約充足問題について解説する．

一般に**制約充足問題**(constraint satisfaction problem)は，以下の3つの要素から構成される．

- 変数(variable): 分からないもの，最適化によって決めるもの．
制約充足問題では，変数は，与えられた集合（以下で述べる「領域」）から1つの要素を選択することによって決められる．

- 領域(domain): 変数ごとに決められた変数の取り得る値の集合

- 制約(constraint): 幾つかの変数が同時にとることのできる値に制限を付加するための条件．
SCOPでは線形制約（線形式の等式，不等式），2次制約（一般の2次式の等式，不等式），
相異制約（集合に含まれる変数がすべて異なることを表す制約）が定義できる．

制約充足問題は，制約をできるだけ満たすように，
変数に領域の中の1つの値を割り当てることを目的とした問題である．


SCOPでは，**重み付き制約充足問題**(weighted constraint satisfaction problem)
を対象とする．

ここで「制約の重み」とは，制約の重要度を表す数値であり，
SCOPでは正数値もしくは無限大を表す文字列 'inf'を入力する．
'inf'を入力した場合には，制約は**絶対制約**(hard constraint)とよばれ，
その逸脱量は優先して最小化される．
重みに正数値を入力した場合には，制約は**考慮制約**(soft constraint)とよばれ，
制約を逸脱した量に重みを乗じたものの和の合計を最小化する．

すべての変数に領域内の値を割り当てたものを**解**(solution)とよぶ．
SCOPでは，単に制約を満たす解を求めるだけでなく，
制約からの逸脱量の重み付き和（ペナルティ）を最小にする解を探索する．

## SCOPの基本クラス

SCOPモジュール (scop.py) は，以下のクラスから構成されている．

- モデルクラス Model
- 変数クラス Variable
- 制約クラス Constraint (これは，以下のクラスのスーパークラスである．）

  - 線形制約クラス Linear
  - 2次制約クラス Quadratic
  - 相異制約クラス Alldiff
 

In [ ]:
#| echo: false
# Image("../figure/scopclass.jpg", width=500)

## 注意

SCOPでは数（制約）名や値を文字列で区別するため重複した名前を付けることはできない．
なお，使用できる文字列は, 英文字 (a--z, A--Z), 
数字 (0--9), 大括弧 ([ ]), アンダーバー (_), および @ に限定される．

それ以外の文字列はすべてアンダーバー (_)に置き換えられる．


## パラメータクラス Parameters

Parametersクラスで設定可能なパラメータは，以下の通り．


-    TimeLimit は制限時間を表す．制限時間は正数値を設定する必要があり，その既定値は 600秒である．

-    OutputFlag は出力フラグを表し，最適化の過程を出力する際の詳細さを制御するためのパラメータである．
真(True もしくは正の値)に設定すると詳細な情報を出力し，
偽(False もしくは 0)に設定すると最小限の情報を出力する．
既定値は偽(0)である．

-    RandomSeed は乱数の種である．SCOPでは探索にランダム性を加味しているので，乱数の種を変えると，得られる解が変わる可能性がある．
乱数の種の既定値は 1である．

-   Target は制約の逸脱量が目標値以下になったら自動終了させるためのパラメータである．
既定値は 0 である．

-  Initial は，前回最適化の探索を行った際の最良解を初期値とした探索を行うとき True ，それ以外のとき  False を表すパラメータである．
既定値は False である．最良解の情報は，「変数:値」を1行としたテキストとしてファイル名 scop_best_data.txt に保管されている．
このファイルを書き換えることによって，異なる初期解から探索を行うことも可能である．

In [ ]:
#| export
class Parameters(BaseModel):
    """
    SCOP parameter class to control the operation of SCOP.

    - TimeLimit: Limits the total time expended (in seconds). Positive integer. Default = 600.
    - OutputFlag: Controls the output log. Boolean. Default = False.
    - RandomSeed: Sets the random seed number. Integer. Default = 1.
    - Target: Sets the target penalty value;
            optimization will terminate if the solver determines that the optimum penalty value
            for the model is worse than the specified "Target." Non-negative integer. Default = 0.
    - Initial: True if you want to solve the problem starting with an initial solution obtained before, False otherwise. Default = False.
    """
    TimeLimit:int    = 600
    OutputFlag: bool = False
    RandomSeed: int  = 1
    Target: int      = 0
    Initial: bool    = False
        
    def __str__(self):
        return f" TimeLimit = {self.TimeLimit} \n OutputFlag = {self.OutputFlag} \n RandomSeed = {self.RandomSeed} \n Taeget = {self.Target} \n Initial = {self.Initial}"

### Parametersクラスの使用例

In [ ]:
params = Parameters()
params.TimeLimit = 3
print(params)

 TimeLimit = 3 
 OutputFlag = False 
 RandomSeed = 1 
 Taeget = 0 
 Initial = False


## 変数クラス Variable

変数クラス Variable のインスタンスは，モデルインスタンスの addVariable もしくは addVariables メソッドを
用いて生成される．

```python
  変数インスタンス=model.addVariable(name, domain）
```

引数の  name は変数名を表す文字列であり，
  domain は領域を表すリストである．

```python
  変数インスタンスのリスト=model.addVariables(names, domain）
```

引数の  names は変数名を要素としたリストであり，  domain は領域を表すリストである．


変数クラスは，以下の属性をもつ．


-    name は変数の名称である．
-    domain は変数の領域(domain)を表すリストである．変数には領域に含まれる値(value)のうちの1つが割り当てられる．
-    value は最適化によって変数に割り当てられた値である．最適化が行われる前には  None が代入されている．


また，変数インスタンスは，変数の情報を文字列として返すことができる．


In [ ]:
#| export
class Variable(BaseModel):
    """
    SCOP variable class. Variables are associated with a particular model.
    You can create a variable object by adding a variable to a model (using Model.addVariable or Model.addVariables)
    instead of by using a Variable constructor.
    """
    ID: ClassVar[int] = 0 #variable ID for anonymous variables
    name: str                    = ""
    domain: List[Union[str,int]] = None
    value: Optional[str]         = None

    def __init__(self, name="", domain = None):
        super().__init__(name = name, domain = domain)
        
        if name is None or name=="":
            name ="__x{0}".format(Variable.ID)
            Variable.ID +=1
        if type(name) != str:
            raise ValueError("Variable name must be a string")
        if domain is None:
            domain = []
            
        #convert illegal characters into _ (underscore)
        self.name   = str(name).translate( _trans )
        #list(domain); domain name is converted to a string
        self.domain = [str(d) for d in domain]
        self.value  = None #optimal value

    def __str__(self):
        return "variable {0}:{1} = {2}".format(
            str(self.name), str(self.domain), str(self.value)
            )

### Variableクラスの使用例


In [ ]:
#標準的な使用法
var = Variable(name="X[1]", domain=[1,2,3])
print(var)

variable X[1]:['1', '2', '3'] = None


In [ ]:
#無記名の例
var1 = Variable(domain = [1,2,3])
var2 = Variable(domain = [4,5,6])
print(var1)
print(var2)

variable __x0:['1', '2', '3'] = None
variable __x1:['4', '5', '6'] = None


In [ ]:
#変数名に数字をいれた場合
try:
    var1 = Variable(name = 1, domain = [1,2,3])
except ValueError as error:
    print(error)

1 validation error for Variable
name
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.6/v/string_type


## 制約クラス Constraint

制約クラスは，以下で定義する線形制約クラス，2次制約クラス，相異制約クラスの基底クラスである．

In [ ]:
#| export
class Constraint(BaseModel):
    """
     Constraint base class
    """
    ID: ClassVar[int] = 0 
    name: Optional[str]   = ""
    weight: Optional[int] = 1
    
    def __init__(self, name="", weight = 1):
        super().__init__(name = name, weight = weight)
        if name is None or name=="":
            name="__CON[{0}]".format(Constraint.ID)
            Constraint.ID+=1
        if type(name) != str:
            raise ValueError("Constraint name must be a string")
        #convert illegal characters into _ (underscore)
        self.name   = str(name).translate( _trans )

    def setWeight(self, weight):
        self.weight = weight

## モデルクラス Model

PythonからSCOPをよび出して使うときに，最初にすべきことはモデルクラス Model のインスタンスを生成することである．
たとえば，  'test' と名付けたモデルインスタンス  model を生成したいときには，以下のように記述する．

```python
from scop import *
model = Model('test')
```

インスタンスの引数はモデル名であり，省略すると無名のモデルが生成される．

Model クラスは，以下のメソッドをもつ．


-    addVariable(name，domain) はモデルに1つの変数を追加する．
引数の name は変数名を表す文字列であり，domain は領域を表すリストである．
変数名を省略すると，自動的に  __x[ 通し番号  ] という名前が付けられる．

領域を定義するためのリスト domain の要素は，文字列でも数値でもかまわない．
（ただし内部表現は文字列であるので，両者は区別されない．）

以下に例を示す．
```python
x = model.addVarriable('var')                     # domain  is set to [] 
x = model.addVariable(name='var',domain=[1,2,3])  # arguments by name
x = model.addVariable('var',['A','B','C'])        # arguments by position
```

1行目の例では，変数名を  'var' と設定した空の領域をもつ変数を追加している．
2行目の例では，名前付き引数で変数名と領域を設定している．領域は  1,2,3 の数値である．
3行目の例では，領域を文字列として変数を追加している．


-    addVariables(names,domain) はモデルに，同一の領域をもつ複数の変数を同時に追加する．
引数の  names は変数名を要素としたリストであり，domain は領域を表すリストである．
領域を定義するためのリストの要素は，文字列でも数値でもかまわない．
        
-    addConstriant(con) は制約インスタンス  con をモデルに追加する．
制約インスタンスは，制約クラスを用いて生成されたインスタンスである．制約インスタンスの生成法については，
以下で解説する．

-    optimize はモデルの求解（最適化）を行うメソッドである．
最適化のためのパラメータは，パラメータ属性  Params で設定する．
返値は，最適解の情報を保管した辞書と，破った制約の情報を保管した辞書のタプルである．

たとえば，以下のプログラムでは最適解を辞書  sol に，破った制約を辞書  violated に保管する．

```python
sol,violated= model.optimize()
```

最適解や破った制約は，変数や制約の名前をキーとし，解の値や制約逸脱量を値とした辞書であるので，
最適解の値と逸脱量を出力するには，以下のように記述すれば良い．

```python
for x in sol:
    print(x, sol[x])
for v in violated:
    print(v,violated[v])
```

　引数：
 
　- cloud: 複数人が同時実行する可能性があるときTrue（既定値はFalse）; Trueのとき，ソルバー呼び出し時に生成されるファイルにタイムスタンプを追加し，計算終了後にファイルを消去する．


モデルインスタンスは，以下の属性をもつ．

-    name はモデルの名前である．コンストラクタの引数として与えられる．省略可で既定値は ' ' である．
-    variables は変数インスタンスのリストである．
-    constraints は制約インスタンスのリストである．
-    varDict は制約名をキーとし，変数インスタンスを値とした辞書である．
-    Params は求解（最適化）の際に用いるパラメータを表す属性を保管する．
-    Status は最適化の状態を表す整数である．状態の種類と意味を以下の表に示す．


最適化の状態を表す整数と意味

|  状態の定数   |  説明  |
| ---- | ---- |
|0                |  最適化成功  |   
|1   |   求解中にユーザが  Ctrl-C を入力したことによって強制終了した．  | 
|2   |   入力データファイルの読み込みに失敗した．    | 
|3   |   初期解ファイルの読み込みに失敗した．  | 
|4   |   ログファイルの書き込みに失敗した．  | 
|5   |  入力データの書式にエラーがある．  | 
|6   |  メモリの確保に失敗した． | 
|7   |  実行ファイル  scop.exe のよび出しに失敗した．  |  
|10   |  モデルの入力は完了しているが，まだ最適化されていない． |  
|負の値  |  その他のエラー  | 


    
また，モデルインスタンスは，モデルの情報を文字列として返すことができる．

In [ ]:
#| export
class Model(BaseModel):
    """
    SCOP model class.

    Attbibutes:
    - constraints: Set of constraint objects in the model.
    - variables: Set of variable objects in the model.
    - Params:  Object including all the parameters of the model.
    - varDict: Dictionary that maps variable names to the variable object.

    """
    name: Optional[str]             = ""
    constraints: Optional[List[Constraint]] = [] # set of constraints is maintained by a list
    variables: Optional[List[Variable]]      = []   # set of variables is maintained by a list
    Params: Optional[Parameters] =Parameters()
    varDict: Optional[Dict[str,List]] ={}      # dictionary that maps variable names to their domains
    Status: Optional[int]                    = 10      # unsolved
    
    def __str__(self):
        """
            return the information of the problem
            constraints are expanded and are shown in a readable format
        """
        ret = ["Model:"+str(self.name) ]
        ret.append( "number of variables = {0} ".format(len(self.variables)) )
        ret.append( "number of constraints= {0} ".format(len(self.constraints)) )
        for v in self.variables:
            ret.append(str(v))

        for c in self.constraints:
            ret.append("{0} :LHS ={1} ".format(str(c)[:-1], str(c.lhs)) )
        return " \n".join(ret)

    def update(self):
        """
        prepare a string representing the current model in the scop input format
        """
        f  = [ ]
        #variable declarations
        for var in self.variables:
            domainList = ",".join([str(i) for i in var.domain])
            f.append( "variable %s in { %s } \n" % (var.name, domainList) )
        #target value declaration
        f.append( "target = %s \n" % str(self.Params.Target) )
        #constraint declarations
        for con in self.constraints:
            f.append(str(con))
        return " ".join(f)

    def addVariable(self, name="", domain=[]):
        """
        - addVariable ( name="", domain=[] )
          Add a variable to the model.

        Arguments:
        - name: Name for new variable. A string object.
        - domain: Domain (list of values) of new variable. Each value must be a string or numeric object.

        Return value:
        New variable object.

        Example usage:
        x = model.addVarriable("var")                     # domain  is set to []
        x = model.addVariable(name="var",domain=[1,2,3])  # arguments by name
        x = model.addVariable("var",["A","B","C"])        # arguments by position

        """
        var =Variable(name,domain)
        # keep variable names using the dictionary varDict
        # to check the validity of constraints later
        # check the duplicated name
        if var.name in self.varDict:
            raise ValueError("duplicate key '{0}' found in variable name".format(var.name))
        else:
            self.variables.append(var)
            self.varDict[var.name]=var
        return var

    def addVariables(self, names=[], domain=[]):
        """
        - addVariables(names=[], domain=[])
           Add variables and their (identical) domain.

        Arguments:
        - names: list of new variables. A list of string objects.
        - domain: Domain (list of values) of new variables. Each value must be a string or numeric object.

        Return value:
        List of new variable objects.

        Example usage:
        varlist=["var1","var2","var3"]
        x = model.addVariables(varlist)                      # domain  is set to []
        x = model.addVariables(names=varlist,domain=[1,2,3]  # arguments by name
        x = model.addVariables(varlist,["A","B","C"]         # arguments by position

        """
        if type(names)!=type([]):
            raise TypeError("The first argument (names) must be a list.")
        varlist=[]
        for var in names:
            varlist.append(self.addVariable(var,domain))
        return varlist

    def addConstraint(self, con):
        """
        addConstraint ( con )
        Add a constraint to the model.

        Argument:
        - con: A constraint object (Linear, Quadratic or AllDiff).

        Example usage:
        model.addConstraint(L)

        """
        if not isinstance(con,Constraint):
            raise TypeError("error: %r should be a subclass of Constraint" % con)

        #check the feasibility of the constraint added in the class con
        try:
            if con.feasible(self.varDict):
                self.constraints.append(con)
        except NameError:
            raise  NameError("Consrtaint %r has an error " % con )

##    def addConstraints(self,*cons):
##        for c in cons:
##            self.addConstraint(c)

    def optimize(self, cloud=False):
        """
        optimize ()
        Optimize the model using scop.exe in the same directory.

        Example usage:
        model.optimize()
        """

        time=self.Params.TimeLimit
        seed=self.Params.RandomSeed
        LOG=self.Params.OutputFlag

        f = self.update()
        p = pathlib.Path(".") #現在のフォルダ
        
        if cloud:
            input_file_name = f"scop_input{dt.datetime.now().timestamp()}.txt"
            f3 = open(input_file_name, "w")
            script = p / "scripts/scop"    
        else:
            f3 = open("scop_input.txt","w")
            script = "./scop"
            
        f3.write(f)
        f3.close()

        if LOG>=100:
            print("scop input: \n")
            print(f)
            print("\n")
        if LOG:
            print("solving using parameters: \n ")
            print("  TimeLimit =%s second \n"%time)
            print("  RandomSeed= %s \n"%seed)
            print("  OutputFlag= %s \n"%LOG)
            
            
        import subprocess
        if platform.system() == "Windows":
            cmd = "scop -time "+str(time)+" -seed "+str(seed) #solver call for win
        elif platform.system()== "Darwin":
            if platform.mac_ver()[2]=="arm64": #M1
                cmd = f"{script}-m1 -time "+str(time)+" -seed "+str(seed)
            else:
                cmd = f"{script} -time "+str(time)+" -seed "+str(seed) #solver call for mac
        elif platform.system() == "Linux":
            cmd = f"{script}-linux -time "+str(time)+" -seed "+str(seed) #solver call for linux
            # github action でエラーするため消しておく
            #exe_file = p / "scripts/scop-linux"
            #os.chmod(exe_file, 0o775)
            
        if self.Params.Initial:
            cmd += " -initsolfile scop_best_data.txt"

        try:
            if platform.system() == "Windows": #Winの場合にはコマンドをsplit!
                pipe = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stdin=subprocess.PIPE, shell=True)
            else:
                pipe = subprocess.Popen(cmd, stdout=subprocess.PIPE, stdin=subprocess.PIPE, shell=True)
            print("\n ================ Now solving the problem ================ \n")
            
            out, err = pipe.communicate(f.encode()) #get the result
            if out ==b"":
                raise OSError
                
        except OSError:
            print("error: could not execute command")
            print("please check that the solver is in the path")
            self.Status = 7  #execution falied
            return None, None

        
        if err!=None:
            if int(sys.version_info[0])>=3:
                err = str(err, encoding='utf-8')
            f2 = open("scop_error.txt","w")
            f2.write(err)
            f2.close()

        if int(sys.version_info[0])>=3:
            out = str(out, encoding='utf-8')

        if cloud:
            os.remove(input_file_name)
            
        if LOG:
            print (out, '\n')
        #print ("out=",out)
        #print ("err=",err)
        #print("Return Code=",pipe.returncode)

        if cloud:
            pass
        else:
            f = open("scop_out.txt","w")
            f.write(out)
            f.close()

        #check the return code
        self.Status = pipe.returncode
        if self.Status !=0: #if the return code is not "optimal", then return
            print("Status=",self.Status)
            print("Output=",out)
            return None, None

        #extract the solution and the violated constraints
        s0 = "[best solution]"
        s1 = "penalty"
        s2 = "[Violated constraints]"
        i0 = out.find(s0) + len(s0)
        i1 = out.find(s1, i0)
        i2 = out.find(s2, i1) + len(s2)

        data = out[i0:i1].strip()

        #save the best solution
        if cloud:
            pass
        else:
            f3 = open("scop_best_data.txt","w")
            f3.write(data.lstrip())
            f3.close()

        sol = {}
        if data != "":
            for s in data.split("\n"):
                name, value = s.split(":")
                sol[name]=value.strip() #remove redunant string

        data = out[i2:].strip()
        violated = {}
        if data != "":
            for s in data.split("\n"):
                try:
                    name, value = s.split(":")
                except:
                    print("Error String=",s)

                try:
                    temp=int(value)
                except:
                    violated[name] = value
                else:
                    violated[name] = int(value)

        #set the optimal solution to the variable
        for name in sol:
            if name in self.varDict:
                self.varDict[name].value = sol[name]
            else:
                raise NameError("Solution {0} is not in variable list".format(name))

        #evaluate the left hand sides of the constraints
        for con in self.constraints:

            if isinstance(con,Linear):
                lhs=0
                for (coeff,var,domain) in con.terms:
                    if var.value==domain:
                        lhs+=coeff

                con.lhs=lhs
            if isinstance(con,Quadratic):
                lhs=0
                #print con.terms
                for (coeff,var1,domain1,var2,domain2) in con.terms:
                    if var1.value==domain1 and var2.value==domain2:
                        lhs+=coeff

                con.lhs=lhs
            if isinstance(con,Alldiff):
                VarSet=set([])
                lhs=0
                for v in con.varlist:
                    index=v.domain.index(v.value)
                    #print v,index
                    if index in VarSet:
                        lhs+=1
                    VarSet.add(index)
                #print VarSet
                con.lhs=lhs
        #return dictionaries containing the solution and the violated constraints
        return sol,violated

### Modelクラスの使用例

In [ ]:
model = Model(name='test')
model.addVariable(name="x[1]",domain=[0,1,2])
print(model)
model.Params.TimeLimit= 1
sol, violated = model.optimize()
print("solution=", sol)
print("violated constraints=", violated)
model.Status

Model:test 
number of variables = 1  
number of constraints= 0  
variable x[1]:['0', '1', '2'] = None

 ================ Now solving the problem ================ 

solution= {'x[1]': '0'}
violated constraints= {}


0

## 線形制約クラス Linear

最も基本的な制約は，線形制約である．
線形制約は
$$
  線形項1 + 線形項2 + \cdots  制約の方向 (\leq, \geq, =) 右辺定数
$$
の形で与えられる．線形項は，「変数」が「値」をとったとき $1$，
それ以外のとき $0$ を表す**値変数**(value variable) $x[変数,値]$ を用いて，
$$
 係数 \times x[変数,値] 
$$
で与えられる．

ここで，係数ならびに右辺定数は整数とし，制約の方向は
以下($\leq$)，以上($\geq$)，等しい($=$)から1つを選ぶ必要がある．

線形制約クラス  Linear のインスタンスは，以下のように生成する．

```python
線形制約インスタンス=Linear(name, weight=1, rhs=0, direction='<=')
```


引数の意味は以下の通り．


-     name は制約の名前を表す．これは制約を区別するための名称であり，固有の名前を文字列で入力する必要がある．
（名前が重複した場合には，前に定義した制約が無視される．）
名前を省略した場合には，自動的に  __CON[ 通し番号  ] という名前が付けられる．
これは，以下の2次制約や相異制約でも同じである．

-    weight は制約の重みを表す．重みは，制約の重要性を表す正数もしくは文字列 'inf' である．
ここで 'inf' は無限大を表し，絶対制約を定義するときに用いられる．
重みは省略することができ，その場合の既定値は 1である．

-    rhs は制約の右辺定数(right hand side)を表す．
右辺定数は，制約の右辺を表す定数（整数値）である．
右辺定数は省略することができ，その場合の既定値は 0 である．

-    direction は制約の向きを表す．
制約の向きは，
  '<=' ,   '>=' ,   '=' のいずれかの文字列とする．既定値は  '<=' である．

上の引数はすべて Linear クラスの属性となる．最適化を行った後では，制約の左辺の評価値が属性として参照可能になる．


-    lhs は制約の左辺の値を表す．これは最適化によって得られた変数の値を左辺に代入したときの評価値である．
最適化を行う前には 0 が代入されている



  Linear クラスは，以下のメソッドをもつ．


-    addTerms(coeffs, vars，values) は，線形制約の左辺に1つもしくは複数の項を追加する．

  addTerms メソッドの引数の意味は以下の通り．
   -    coeffs は追加する項の係数もしくは係数リスト．係数もしくはリストの要素は整数．
   -    vars は追加する項の変数インスタンスもしくはの変数インスタンスのリスト．リストの場合には，リストcoeffsと同じ長さをもつ必要がある．
   -    values は追加する項の値もしは値のリスト．リストの場合には，リストcoeffsと同じ長さをもつ必要がある．
　
  
  addTerms メソッドは，1つの項を追加するか，複数の項を一度に追加する．1つの項を追加する場合には，引数の係数は整数値，変数は変数インスタンスで与え，値は変数の領域の要素とする．複数の項を一度に追加する場合には，同じ長さをもつ，係数，変数インスタンス，値のリストで与える．

たとえば，項をもたない線形制約インスタンス L に対して，
```
L.addTerms(1, y, 'A')
```
と1つの項を追加すると，制約の左辺は
```
1 x[y, 'A'] 
```
となる．ここで  x は値変数（ y が 'A' になるとき 1，それ以外のとき 0 の仮想の変数）を表す．

同様に，項をもたない線形制約インスタンス  L に対して，
```
L.addTerms([2, 3, 1], [y, y, z], ['C', 'D', 'C']) 
```
と3つの項を同時に追加すると，制約の左辺は以下のようになる．
```
 2 x[y,'C']  + 3 x[y,'D'] + 1 x[z,'C']
```
-    setRhs(rhs) は線形制約の右辺定数を  rhs に設定する．引数は整数値であり，既定値は 0 である．

-    setDirection(dir) は制約の向きを設定する．引数  dir は
  '<=' ,   '>=' ,   '=' のいずれかの文字列とする．既定値は  '<=' である．

-    setWeight(weight) は制約の重みを  weight に設定する．引数は正数値もしくは文字列  'inf' である．
ここで  'inf' は無限大を表し，絶対制約を定義するときに用いられる．

また，線形制約クラス  Linear は，制約の情報を文字列として返すことができる．


In [ ]:
#| export
class Linear(Constraint):
    """
    Linear ( name, weight=1, rhs=0, direction="<=" )
    Linear constraint constructor.

    Arguments:
    - name: Name of linear constraint.
    - weight (optiona): Positive integer representing importance of constraint.
    - rhs: Right-hand-side constant of linear constraint.
    - direction: Rirection (or sense) of linear constraint; "<=" (default) or ">=" or "=".

    Attributes:
    - name: Name of linear constraint.
    - weight (optional): Positive integer representing importance of constraint.
    - rhs: Right-hand-side constant of linear constraint.
    - lhs: Left-hand-side constant of linear constraint.
    - direction: Direction (or sense) of linear constraint; "<=" (default) or ">=" or "=".
    - terms: List of terms in left-hand-side of constraint. Each term is a tuple of coeffcient,variable and its value.
    """
    name: str                 = ""
    weight: Optional[Union[int,str]]     = 1
    rhs: Optional[int]        = 0
    direction: Optional[str]  = "<="
    terms: Optional[List[Tuple[int,Variable,str]]] = []
    lhs: int                  = 0
    
    def __init__(self, name="", weight=1, rhs=0, direction="<="):
        """
        Constructor of linear constraint class:
        """
        super(Linear, self).__init__(name = name, weight = weight)
        
        if direction in ["<=", ">=", "="]:
            self.direction = direction
        else:
            raise NameError("direction setting error;direction should be one of '<=', '>=', or '='")


    def __str__(self):
        """ 
            return the information of the linear constraint
            the constraint is expanded and is shown in a readable format
        """
        f =["{0}: weight= {1} type=linear".format(self.name, self.weight)]
        for (coeff,var,value) in self.terms:
            f.append( "{0}({1},{2})".format(str(coeff),var.name,str(value)) )
        f.append( self.direction+str(self.rhs) +"\n" )
        return " ".join(f)

    def addTerms(self,coeffs=[],vars=[],values=[]):
        """
            - addTerms ( coeffs=[],vars=[],values=[] )
            Add new terms into left-hand-side of linear constraint.

            Arguments:
            - coeffs: Coefficients for new terms; either a list of coefficients or a single coefficient. The three arguments must have the same size.
            - vars: Variables for new terms; either a list of variables or a single variable. The three arguments must have the same size.
            - values: Values for new terms; either a list of values or a single value. The three arguments must have the same size.

            Example usage:

            L.addTerms(1, y, "A")
            L.addTerms([2, 3, 1], [y, y, z], ["C", "D", "C"]) #2 X[y,"C"]+3 X[y,"D"]+1 X[z,"C"]
        """
        if type(coeffs) !=type([]): #need a check whether coeffs is numeric ...
            #arguments are not a list; add a term
            if type(coeffs)==type(1):  #整数の場合だけ追加する．
                self.terms.append( (coeffs,vars,str(values)))
            else:
                raise ValueError("Coefficient must be an integer.")
        elif type(coeffs)!=type([]) or type(vars)!=type([]) or type(values)!=type([]):
            raise TypeError("coeffs, vars, values must be lists")
        elif len(coeffs)!=len(vars) or len(coeffs)!=len(values):
            raise TypeError("length of coeffs, vars, values must be identical")
        else:
            for i in range(len(coeffs)):
                self.terms.append( (coeffs[i],vars[i],str(values[i])))

    def setRhs(self,rhs=0):
        if type(rhs) != type(1):
            raise ValueError("Right-hand-side must be an integer.")
        else:
            self.rhs = rhs

    def setDirection(self,direction="<="):
        if direction in ["<=",">=","="]:
            self.direction = direction
        else:
            raise NameError(
                "direction setting error; direction should be one of '<=', '>=', or '='"
                           )

    def feasible(self,allvars):
        """ 
        return True if the constraint is defined correctly
        """
        for (coeff,var,value) in self.terms:
            if var.name not in allvars:
                raise NameError("no variable in the problem instance named %r" % var.name)
            if value not in allvars[var.name].domain:
                raise NameError("no value %r for the variable named %r" % (value, var.name))
        return True

### Linearクラスの使用例

In [ ]:
# 通常の制約
L = Linear(name = "a linear constraint", weight="inf", rhs = 10, direction = "<=" )
x = Variable(name="x", domain=["A","B","C"] )
L.addTerms(3, x, "A")
print(L)

a_linear_constraint: weight= inf type=linear 3(x,A) <=0



In [ ]:
#右辺定数や制約の係数が整数でない場合
try:
    L = Linear(name = "a linear constraint", rhs = 10.56, direction = "=" )
except ValueError as error:
    print(error)
    
x = Variable(name="x", domain=["A","B","C"] )
try:
    L.addTerms(3.1415, x, "A")
except ValueError as error:
    print(error)

Coefficient must be an integer.


## 2次制約クラス Quadraric

SCOPでは（非凸の）2次関数を左辺にもつ制約（2次制約）も扱うことができる．

2次制約は，
$$
  2次項1 + 2次項2 + \cdots  制約の方向 (\leq, \geq, =) 右辺定数
$$
の形で与えられる．ここで2次項は，
$$
 係数 \times x[変数1,値1] \times  x[変数2,値2] 
$$
で与えられる．


2次制約クラス  Quadratic のインスタンスは，以下のように生成する．

```python
2次制約インスタンス=Quadratic(name, weight=1, rhs=0, direction='<=')
```

2次制約クラスの引数と属性は，線形制約クラス  Linear と同じである．

Quadratic クラスは，以下のメソッドをもつ．

```python
addTerms(coeffs,vars1,values1,vars2,values2) は2次制約の左辺に2つの変数の積から成る項を追加する．
```


2次制約に対する addTerms メソッドの引数は以下の通り．

-    coeffs は追加する項の係数もしくは係数のリスト．係数もしくはリストの要素は整数．
-    vars1 は追加する項の第1変数インスタンスもしくは変数インスタンスのリスト． リストの場合には，リスト  coeffs と同じ長さをもつ必要がある．
-    values1 は追加する項の第1変数の値もしくは値のリスト． リストの場合には，リスト coeffs と同じ長さをもつ必要がある．
-    vars2 は追加する項の第2変数の変数インスタンスもしくは変数インスタンスのリスト． リストの場合には，リスト  coeffs と同じ長さをもつ必要がある．
-    values2 は追加する項の第2変数の値もしくは値のリスト． リストの場合には， リスト coeffs と同じ長さをもつ必要がある．

addTerms メソッドは，1つの項を追加するか，複数の項を一度に追加する．
1つの項を追加する場合には，
引数の係数は整数値，変数は変数インスタンスで与え，値は変数の領域の要素とする．
複数の項を一度に追加する場合には，同じ長さをもつ，係数，変数インスタンス，値のリストで与える．

たとえば，項をもたない2次制約インスタンス  Q に対して，
```python
Q.addTerms(1, y, 'A', z, 'B')       
```
と1つの項を追加すると，制約の左辺は
```python
1 x[y,'A'] * x[z,'B'] 
```
となる．

同様に，項をもたない2次制約インスタンス  Q に対して，
```python
Q.addTerms([2, 3, 1], [y, y, z], ['C', 'D', 'C'], [x, x, y], ['A', 'B', 'C'])
```
と3つの項を同時に追加すると，制約の左辺は以下のようになる．
```python
2 x[y,'C'] * x[x,'A'] + 3 x[y,'D'] * x[x,'B'] + 1 x[z,'C'] * x[y,'C']
```

-   setRhs(rhs) は2次制約の右辺定数を  rhs に設定する．引数は整数値であり，既定値は 0 である．

-    setDirection(dir) は制約の向きを設定する．引数  dir は
  '<=' ,   '>=' ,   '=' のいずれかの文字列とする．既定値は  '<=' である．

-    setWeight(weight) は制約の重みを設定する．引数は正数値もしくは文字列  'inf' である．
  'inf' は無限大を表し，絶対制約を定義するときに用いられる．

また，2次制約クラス  Quadratic は，制約の情報を文字列として返すことができる．

In [ ]:
#| export
class Quadratic(Constraint):
    """
    Quadratic ( name, weight=1, rhs=0, direction="<=" )
    Quadratic constraint constructor.

    Arguments:
    - name: Name of quadratic constraint.
    - weight (optional): Positive integer representing importance of constraint.
    - rhs: Right-hand-side constant of linear constraint.
    - direction: Direction (or sense) of linear constraint; "<=" (default) or ">=" or "=".

    Attributes:
    - name: Name of quadratic constraint.
    - weight (optiona): Positive integer representing importance of constraint.
    - rhs: Right-hand-side constant of linear constraint.
    - lhs: Left-hand-side constant of linear constraint.
    - direction: Direction (or sense) of linear constraint; "<=" (default) or ">=" or "=".
    - terms: List of terms in left-hand-side of constraint. Each term is a tuple of coeffcient, variable1, value1, variable2 and value2.
    """

    name: str                 = ""
    weight: Optional[Union[int,str]]     = 1
    rhs: Optional[int]        = 0
    direction: Optional[str]  = "<="
    terms: Optional[List[Tuple[int,Variable,str,Variable,str]]] = []
    lhs: int                  = 0
    
    def __init__(self, name="", weight=1, rhs=0, direction="<="):
        super(Quadratic,self).__init__(name=name, weight=weight)

        if direction in ["<=", ">=", "="]:
            self.direction = direction
        else:
            raise NameError(
                "direction setting error;direction should be one of '<=', '>=', or '='"
                  )


    def __str__(self):
        """ return the information of the quadratic constraint
            the constraint is expanded and is shown in a readable format
        """
        f = [ "{0}: weight={1} type=quadratic".format(self.name,self.weight) ]
        for (coeff,var1,value1,var2,value2) in self.terms:
            f.append( "{0}({1},{2})({3},{4})".format(
                str(coeff),var1.name,str(value1),var2.name,str(value2)
                ))
        f.append( self.direction+str(self.rhs) +"\n" )
        return " ".join(f)

    def addTerms(self,coeffs=[],vars=[],values=[],vars2=[],values2=[]):
        """
        addTerms ( coeffs=[],vars=[],values=[],vars2=[],values2=[])

        Add new terms into left-hand-side of qua
        dratic constraint.

        Arguments:
        - coeffs: Coefficients for new terms; either a list of coefficients or a single coefficient. The five arguments must have the same size.
        - vars: Variables for new terms; either a list of variables or a single variable. The five arguments must have the same size.
        - values: Values for new terms; either a list of values or a single value. The five arguments must have the same size.
        - vars2: Variables for new terms; either a list of variables or a single variable. The five arguments must have the same size.
        - values2: Values for new terms; either a list of values or a single value. The five arguments must have the same size.

        Example usage:

        L.addTerms(1, y, "A", z, "B")

        L.addTerms([2, 3, 1], [y, y, z], ["C", "D", "C"], [x, x, y], ["A", "B", "C"])
                  #2 X[y,"C"] X[x,"A"]+3 X[y,"D"] X[x,"B"]+1 X[z,"C"] X[y,"C"]

        """
        if type(coeffs) !=type([]): 
            if type(coeffs)==type(1):  #整数の場合だけ追加する．
                self.terms.append( (coeffs,vars,str(values),vars2,str(values2)))
            else:
                raise ValueError("Coefficient must be an integer.")
        elif type(coeffs)!=type([]) or type(vars)!=type([]) or type(values)!=type([]) \
             or type(vars2)!=type([]) or type(values2)!=type([]):
            raise TypeError("coeffs, vars, values must be lists")
        elif len(coeffs)!=len(vars) or len(coeffs)!=len(values) or len(values)!=len(vars) \
             or len(coeffs)!=len(vars2) or len(coeffs)!=len(values2):
            raise TypeError("length of coeffs, vars, values must be identical")
        else:
            for i in range(len(coeffs)):
                self.terms.append( (coeffs[i],vars[i],str(values[i]),vars2[i],str(values2[i])))

    def setRhs(self,rhs=0):
        if type(rhs) != type(1):
            raise ValueError("Right-hand-side must be an integer.")
        else:
            self.rhs = rhs

    def setDirection(self,direction="<="):
        if direction in ["<=", ">=", "="]:
            self.direction = direction
        else:
            raise NameError(
                "direction setting error;direction should be one of '<=', '>=', or '='"
                  )

    def feasible(self,allvars):
        """
          return True if the constraint is defined correctly
        """
        for (coeff,var1,value1,var2,value2) in self.terms:
            if var1.name not in allvars:
                raise NameError("no variable in the problem instance named %r" % var1.name)
            if var2.name not in allvars:
                raise NameError("no variable in the problem instance named %r" % var2.name)
            if value1 not in allvars[var1.name].domain:
                raise NameError("no value %r for the variable named %r" % (value1, var1.name))
            if value2 not in allvars[var2.name].domain:
                raise NameError("no value %r for the variable named %r" % (value2, var2.name))
        return True

### Quadraticクラスの使用例

In [ ]:
Q = Quadratic(name = "a quadratic constraint", rhs = 10, direction = "<=" )
x = Variable(name="x",domain=["A","B","C"] )
y = Variable(name="y",domain=["A","B","C"] )
Q.addTerms([3,9], [x,x], ["A","B"], [y,y], ["B","C"])
print(Q)

a_quadratic_constraint: weight=1 type=quadratic 3(x,A)(y,B) 9(x,B)(y,C) <=0



## 相異制約クラス Alldiff

相異制約は，変数の集合に対し, 集合に含まれる変数すべてが異なる値を
とらなくてはならないことを規定する．
これは組合せ的な構造に対する制約であり，制約最適化の特徴的な制約である．

SCOPにおいては，値が同一であるかどうかは，値の名称ではなく，
変数のとりえる値の集合（領域）を表したリストにおける**順番（インデックス）**によって決定される.
たとえば, 変数  var1 および  var2 の領域がそれぞれ  ['A','B'] ならびに  ['B','A'] であったとき，
変数  var1 の値   'A', 'B'  の順番はそれぞれ 0 と 1， 
変数  var2 の値   'A', 'B'  の順番はそれぞれ 1 と 0 となる．
したがって，**相異制約を用いる際には変数に同じ領域を与える**ことが（混乱を避けるという意味で）推奨される．

相異制約クラス  Alldiff のインスタンスは，以下のように生成する．

```python
相異制約インスタンス = Alldiff(name, varlist, weight)
```

引数の名前と既定値は以下の通り．


-    name は制約名を与える．

-    varlist は相異制約に含まれる変数インスタンスのリストを与える．
これは，値の順番が異なることを要求される変数のリストであり，省略も可能である．
その場合の既定値は，空のリストとなる．
ここで追加する変数は，モデルクラスに追加された変数である必要がある．

-    weight は制約の重みを与える．

相異制約の制約名と重みについては，線形制約クラス  Linear と同じように設定する．
上の引数は  Alldiff クラスの属性でもある．その他の属性として最適化した後で得られる式の評価値がある．


-    lhs は左辺(left hand side)の評価値を表し，最適化された後に，同じ値の番号（インデックス）をもつ変数の数が代入される．


  Alldiff クラスは，以下のメソッドをもつ．


-    addVariable(var) は相異制約に1つの変数インスタンス  var を追加する．

-    addVariables(varlist) は相異制約の変数インスタンスを複数同時に（リスト  varlist として）追加する．

-    setWeight(weight) は制約の重みを設定する．引数は正数値もしくは文字列  'inf' である．  'inf' は無限大を表し，絶対制約を定義するときに用いられる．


また，相異制約クラス  Alldiff は，制約の情報を文字列として返すことができる．

In [ ]:
#| export
class Alldiff(Constraint):
    """
    Alldiff ( name=None,varlist=None,weight=1 )
    Alldiff type constraint constructor.

    Arguments:
    - name: Name of all-different type constraint.
    - varlist (optional): List of variables that must have differennt value indices.
    - weight (optional): Positive integer representing importance of constraint.

    Attributes:
    - name: Name of all-different type  constraint.
    - varlist (optional): List of variables that must have differennt value indices.
    - lhs: Left-hand-side constant of linear constraint.

    - weight (optional): Positive integer representing importance of constraint.
    """

    name: str                 = ""
    weight: Optional[Union[int,str]]     = 1
    varlist: List[Variable]   = []
    lhs: int                  = 0   
    
    def __init__(self, name="", varlist=None, weight=1):
        super(Alldiff,self).__init__(name=name, weight=weight)
        self.lhs=0
        if varlist==None:
            self.varlist = []
        else:
            for var in varlist:
                if not isinstance(var, Variable):
                    raise NameError("error: %r should be a subclass of Variable" % var)
            self.varlist = varlist

    def __str__(self):
        """
        return the information of the alldiff constraint
        """
        f = [ "{0}: weight= {1} type=alldiff ".format(self.name,self.weight) ]
        for var in self.varlist:
            f.append( var.name )
        f.append( "; \n" )
        return " ".join(f)

    def addVariable(self,var):
        """
        addVariable ( var )
        Add new variable into all-different type constraint.

        Arguments:
        - var: Variable object added to all-different type constraint.

        Example usage:

        AD.addVaeiable( x )

        """
        if not isinstance(var, Variable):
            raise NameError("error: %r should be a subclass of Variable" % var)

        if var in self.varlist:
            print("duplicate variable name error when adding variable %r" % var)
            return False
        self.varlist.append(var)

    def addVariables(self, varlist):
        """
        addVariables ( varlist )
        Add variables into all-different type constraint.

        Arguments:
        - varlist: List or tuple of variable objects added to all-different type constraint.

        Example usage:

        AD.addVariables( x, y, z )

        AD.addVariables( [x1,x2,x2] )

        """
        for var in varlist:
            self.addVariable(var)

    def feasible(self, allvars):
        """
           return True if the constraint is defined correctly
        """
        for var in self.varlist:
            if var.name not in allvars:
                raise NameError("no variable in the problem instance named %r" % var.name)
        return True

### Alldiffクラスの使用例

In [ ]:
A = Alldiff(name="a alldiff constraint")
x = Variable(name="x",domain=["A","B","C"] )
y = Variable(name="y",domain=["A","B","C"] )
A.addVariables([x,y])
print(A)

a_alldiff_constraint: weight= 1 type=alldiff  x y ; 



## 最適化の描画関数 plot_scop

SCOPはメタヒューリスティクスによって解の探索を行う． 一般には，解の良さと計算時間はトレードオフ関係がある．つまり，計算時間をかければかけるほど良い解を得られる可能性が高まる． 
どの程度の計算時間をかければ良いかは，最適化したい問題例（問題に数値を入れたもの）による． plot_scopは，横軸に計算時間，縦軸に目的関数値をプロットする関数であり，最適化を行ったあとに呼び出す． 
得られるPlotlyの図は，どのくらいの計算時間をかければ良いかをユーザーが決めるための目安を与える．

たとえば以下の例の図から，500秒程度の計算時間で良好な解を得ることができるが，念入りにさらに良い解を探索したい場合には2000秒以上の計算時間が必要なことが分かる．

In [ ]:
#| export
def plot_scop(file_name: str="scop_out.txt"):
    with open(file_name) as f:
        out = f.readlines()
    x, y1, y2 = [],[],[] 
    for l in out[5:]: 
        sep = re.split("[=()/]", l)
        #print(sep)
        if sep[0] == '# penalty ':
            break
        if sep[0] == 'penalty ':
            hard, soft, cpu = map(float, [ sep[1], sep[2], sep[6]])
            x.append(cpu)
            y1.append(hard)
            y2.append(soft)

    fig = go.Figure()
    fig.add_trace(go.Scatter(
            x = x, 
            y = y1,
            mode='markers+lines',
            name= "hard",
            marker=dict(
                size=10,
                color= "red")
    ))
    fig.add_trace(go.Scatter(
            x = x, 
            y = y2,
            name ="soft",
            mode='markers+lines',
            marker=dict(
                size=8,
                color= "blue")
    ))
    fig.update_layout(title = "SCOP performance",
                   xaxis_title='CPU time',
                   yaxis_title='Penalty')
    return fig

In [ ]:
fig = plot_scop()
#plotly.offline.plot(fig);

In [ ]:
model = Model()
# 変数の宣言
A = model.addVariable(name="A", domain=[0, 1, 2])
B = model.addVariable(name="B", domain=[0, 1, 2])
C = model.addVariable(name="C", domain=[0, 1, 2])

# 相異制約
alldiff = Alldiff("All Diff", [A, B, C], weight="inf")
model.addConstraint(alldiff)

# 目的関数
linear = Linear(name="Objective Function", weight=1, rhs=0, direction="<=")
linear.addTerms([15, 20, 30], [A, A, A], [0, 1, 2])
linear.addTerms([7, 15, 12], [B, B, B], [0, 1, 2])
linear.addTerms([25, 10, 13], [C, C, C], [0, 1, 2])
model.addConstraint(linear)

print(model)

# 返値は解を表す辞書と逸脱を表す辞書
sol, violated = model.optimize()
print("solution=", sol)
print("violated constraint=", violated)

Model: 
number of variables = 3  
number of constraints= 2  
variable A:['0', '1', '2'] = None 
variable B:['0', '1', '2'] = None 
variable C:['0', '1', '2'] = None 
All_Diff: weight= inf type=alldiff  A B C ;  :LHS =0  
Objective_Function: weight= 1 type=linear 15(A,0) 20(A,1) 30(A,2) 7(B,0) 15(B,1) 12(B,2) 25(C,0) 10(C,1) 13(C,2) <=0 :LHS =0 

 ================ Now solving the problem ================ 

solution= {'A': '0', 'B': '2', 'C': '1'}
violated constraint= {'Objective_Function': 37}


In [ ]:
model = Model()
workers = ["A", "B", "C", "D", "E"]
Jobs = [0, 1, 2]
Cost = {
    ("A", 0): 15,
    ("A", 1): 20,
    ("A", 2): 30,
    ("B", 0): 7,
    ("B", 1): 15,
    ("B", 2): 12,
    ("C", 0): 25,
    ("C", 1): 10,
    ("C", 2): 13,
    ("D", 0): 15,
    ("D", 1): 18,
    ("D", 2): 3,
    ("E", 0): 5,
    ("E", 1): 12,
    ("E", 2): 17,
}
LB = {0: 1, 1: 2, 2: 2}
x = {}
for i in workers:
    x[i] = model.addVariable(name=i, domain=Jobs)
LBC = {}
for j in Jobs:
    LBC[j] = Linear(f"LB{j}", "inf", LB[j], ">=")
    for i in workers:
        LBC[j].addTerms(1, x[i], j)
    model.addConstraint(LBC[j])
obj = Linear("obj")
for i in workers:
    for j in [0, 1, 2]:
        obj.addTerms(Cost[i, j], x[i], j)
model.addConstraint(obj)

model.Params.TimeLimit = 1
sol, violated = model.optimize()

print("solution")
for x in sol:
    print(x, sol[x])
print("violated constraint(s)")
for v in violated:
    print(v, violated[v])


 ================ Now solving the problem ================ 

solution
A 0
B 0
C 1
D 2
E 0
violated constraint(s)
obj 40


In [ ]:
model = Model()
workers = ["A", "B", "C", "D", "E"]
Jobs = [0, 1, 2]
Cost = {
    ("A", 0): 15,
    ("A", 1): 20,
    ("A", 2): 30,
    ("B", 0): 7,
    ("B", 1): 15,
    ("B", 2): 12,
    ("C", 0): 25,
    ("C", 1): 10,
    ("C", 2): 13,
    ("D", 0): 15,
    ("D", 1): 18,
    ("D", 2): 3,
    ("E", 0): 5,
    ("E", 1): 12,
    ("E", 2): 17,
}
LB = {0: 1, 1: 2, 2: 2}
x = {}
for i in workers:
    x[i] = model.addVariable(i, Jobs)
LBC = {}
for j in Jobs:
    LBC[j] = Linear(f"LB{j}", "inf", LB[j], ">=")
    for i in workers:
        LBC[j].addTerms(1, x[i], j)
    model.addConstraint(LBC[j])
obj = Linear("obj", 1, 0, "<=")
for i in workers:
    for j in Jobs:
        obj.addTerms(Cost[i, j], x[i], j)
model.addConstraint(obj)
conf = Quadratic("conflict", 100, 0, "=")
for j in Jobs:
    conf.addTerms(1, x["A"], j, x["C"], j)
model.addConstraint(conf)
model.Params.TimeLimit = 1
sol, violated = model.optimize()
print("solution")
for x in sol:
    print(x, sol[x])
print("violated constraint(s)")
for v in violated:
    print(v, violated[v])


 ================ Now solving the problem ================ 

solution
A 0
B 0
C 1
D 2
E 0
violated constraint(s)
obj 40
